# PROYECTO 2. RKG (RESTART KERNEL GIRLS)
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import re
import sys

sys.path.append('../')
from src import soporte_variables as spv
from src import soporte_funciones as spf

pd.options.display.max_columns = None

## TRATAMIENTO Y LIMPIEZA DE LOS DATOS A ESTUDIAR

## 1. Unión de las preguntas separadas en varias columnas (q10, q39)

In [2]:
# abrimos el .csv con los datos de Data Analyst para las preguntas relevantes seleccionadas
df = pd.read_csv('../resultados/df_da_relevante.csv', index_col = 0)
df = df.reset_index()
df.sample(2)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
405,4734,60-69,Man,Other,Master’s degree,< 1 years,Python,Python,Visual Studio Code (VSCode),Kaggle Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn",Government/Public Service,50-249 employees,3-4,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Basic statistical software (Microsoft Excel, G..."
1294,14394,18-21,Woman,Brazil,Bachelor’s degree,5-10 years,"Python, R, SQL",Python,"Visual Studio Code (VSCode) , Sublime Text,Jup...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express ...",Accounting/Finance,"1000-9,999 employees",20+,Analyze and understand data to influence produ...,"Oracle Database , Amazon Redshift","Microsoft Power BI, Other",Other,NaN,NaN,NaN,GitHub,NaN,Kaggle,Colab,NaN,NaN,Other,Other


In [3]:
# sacamos los nulos para monitorearlos a lo largo del proceso
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
index,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41
0,0,0,0,0,0,0,143,144,146,1508,1639,2191,2287,2124,2286,2204,2250,2267,2132,2114,2231,2281,2280,2276,1643,2206,193,114,130,150,227,1062,1069,1858,2266,2252,2278,1739,2208,1863,2104,2251,1884,2253,533


In [4]:
# creamos una lista con las respuestas de la q10 unificadas
lista_futura_q10 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q10" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q10.append(valor.strip(','))
    except:
        lista_futura_q10.append(valor)
lista_futura_q10[:5]

['Kaggle Notebooks,Colab Notebooks,Databricks Collaborative Notebooks',
 'Google Cloud Datalab',
 'Kaggle Notebooks,Colab Notebooks',
 'Kaggle Notebooks,Azure Notebooks',
 'IBM Watson Studio']

In [5]:
# creamos una lista con las respuestas de la q39 unificadas
lista_futura_q39 = []

for indice, row in df.iterrows():
    #print(indice, row.index)
    valor = ""
    for i in row.index:
        if "q39" in i:
            if type(row[i]) != float:
                row[i] = row[i].strip()
                valor += row[i] + ","
                if valor == ",":
                    valor = ''
    if valor == '':
        valor = np.nan
    try:
        lista_futura_q39.append(valor.strip(','))
    except:
        lista_futura_q39.append(valor)
lista_futura_q39[:5]

[nan,
 nan,
 'I do not share my work publicly',
 nan,
 'I do not share my work publicly']

In [6]:
# creamos las columnas unificadas en el dataframe y borramos las sobrantes
df['q10'] = lista_futura_q10
df['q39'] = lista_futura_q39
df.head(3)

,index,age,gender,q3,q4,q6,q7,q8,q9,q10_part_1,q10_part_2,q10_part_3,q10_part_4,q10_part_5,q10_part_6,q10_part_7,q10_part_8,q10_part_9,q10_part_10,q10_part_11,q10_part_12,q10_part_13,q10_part_14,q10_part_15,q10_part_16,q10_other,q14,q20,q21,q22,q24,q32,q34,q35,q39_part_1,q39_part_2,q39_part_3,q39_part_4,q39_part_5,q39_part_6,q39_part_7,q39_part_8,q39_part_9,q39_other,q41,q10,q39
0,16,50-54,Man,Belgium,Bachelor’s degree,20+ years,"Python, SQL",Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"1000-9,999 employees",5-9,Analyze and understand data to influence produ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Local development environments (RStudio, Jupyt...","Kaggle Notebooks,Colab Notebooks,Databricks Co...",NaN
1,32,22-24,Nonbinary,United States of America,Some college/university study without earning ...,< 1 years,R,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,250-999 employees,3-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN
2,33,30-34,Woman,Egypt,Bachelor’s degree,3-5 years,Python,R,"Notepad++,Jupyter Notebook",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,Non-profit/Service,0-49 employees,0,Analyze and understand data to influence produ...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I do not share my work publicly,NaN,"Basic statistical software (Microsoft Excel, G...","Kaggle Notebooks,Colab Notebooks",I do not share my work publicly


In [7]:
df.drop(columns=spv.columnas_sobrantes, inplace=True)
df.sample()

,index,age,gender,q3,q4,q6,q7,q8,q9,q14,q20,q21,q22,q24,q32,q34,q35,q41,q10,q39
67,826,22-24,Man,China,Bachelor’s degree,< 1 years,Python,Python,PyCharm,None,Academics/Education,0-49 employees,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [8]:
# reordenamos las columnas
df = df.reindex(columns = ['age', 'gender', 'q3', 'q4', 'q6', 'q7', 'q8', 'q9', 'q10', 'q14', 'q20', 'q21', 'q22', 'q24', 'q32', 'q34', 'q35', 'q39', 'q41'])

In [9]:
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,144,146,151,193,114,130,150,227,1062,1069,1858,1099,533


In [10]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
237,18-21,Man,Russia,Master’s degree,< 1 years,Swift,SQL,Visual Studio,Observable Notebooks,None,Energy/Mining,50-249 employees,3-4,Build and/or run the data infrastructure that...,Google Cloud BigTable,Tableau,NaN,"Kaggle,Colab","Basic statistical software (Microsoft Excel, G..."
689,18-21,Woman,India,Bachelor’s degree,1-3 years,"Python, C++, Java, Javascript, Other",Python,"Visual Studio Code (VSCode) ,Jupyter Notebook",Kaggle Notebooks,"Matplotlib ,Seaborn",Academics/Education,250-999 employees,1-2,Analyze and understand data to influence produ...,MongoDB,None,NaN,"GitHub,Kaggle","Basic statistical software (Microsoft Excel, G..."
1764,25-29,Woman,Iraq,Master’s degree,1-3 years,Python,Python,Visual Studio Code (VSCode),None,None,Retail/Sales,50-249 employees,1-2,Analyze and understand data to influence produ...,None,None,NaN,I do not share my work publicly,"Basic statistical software (Microsoft Excel, G..."


## 2. Tratamiento de las columnas que separaremos con .explode() más adelante

In [11]:
# limpiamos la columna 'q9'
df['q9'] = df['q9'].apply(spf.limpiar_q9)
df['q9'].reset_index().sample(10)

,index,q9
540,540,"Jupyter-JupyterLab-JupyterNotebooks ,Jupyter N..."
944,944,Visual Studio Code (VSCode)
710,710,"Visual Studio Code (VSCode) , Notepad++,Jupyte..."
1085,1085,"RStudio ,PyCharm ,Jupyter Notebook"
1927,1927,"Visual Studio , Spyder,Jupyter Notebook"
297,297,"Visual Studio ,Visual Studio Code (VSCode) , N..."
2008,2008,RStudio
1793,1793,None
1053,1053,"RStudio , Notepad++"
340,340,"RStudio , Notepad++,Jupyter Notebook"


In [12]:
# limpiamos la columna 'q24'
df['q24'] = df['q24'].apply(spf.limpiar_q24)
df['q24'].reset_index().sample(10)

,index,q24
214,214,None of these activities are an important par...
2222,2222,Analyze and understand data to influence produ...
1555,1555,Analyze and understand data to influence produ...
1328,1328,Analyze and understand data to influence produ...
1350,1350,Analyze and understand data to influence produ...
135,135,Analyze and understand data to influence produ...
752,752,None of these activities are an important par...
1233,1233,Analyze and understand data to influence produ...
2127,2127,Analyze and understand data to influence produ...
1730,1730,Analyze and understand data to influence produ...


In [13]:
# limpiamos la columna 'q41'
df['q41'] = df['q41'].apply(spf.limpiar_q41a)
df['q41'] = df['q41'].apply(spf.limpiar_q41b)
df['q41'] = df['q41'].apply(spf.limpiar_q41c)
df['q41'] = df['q41'].apply(spf.limpiar_q41d)
df['q41'] = df['q41'].apply(spf.limpiar_q41e)
df['q24'].reset_index().sample(10)

,index,q24
1905,1905,Analyze and understand data to influence produ...
1320,1320,Analyze and understand data to influence produ...
2284,2284,Analyze and understand data to influence produ...
630,630,Analyze and understand data to influence produ...
855,855,Analyze and understand data to influence produ...
1375,1375,Analyze and understand data to influence produ...
377,377,Analyze and understand data to influence produ...
1426,1426,Analyze and understand data to influence produ...
2000,2000,Analyze and understand data to influence produ...
1341,1341,Analyze and understand data to influence produ...


In [14]:
# hacemos un split en las columnas en las que más adelante haremos un .explode()
for col in spv.columnas_unidas:
    df[col] = df[col].str.split(',')

In [15]:
#chequeamos los nulos
df.isnull().sum().reset_index().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
index,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41
0,0,0,0,0,0,143,144,146,151,193,114,130,150,227,1062,1069,1858,1099,533


## 3. Creación de una nueva columna para agrupar los paises por continentes

In [16]:
# creamos un diccionario que asocie cada país con su continente correspondiente
# con él creamos una nueva columna con los continentes correspondientes a cada país
df['continente'] = df['q3'].map(spv.continentes)

In [17]:
df.sample(3)

,age,gender,q3,q4,q6,q7,q8,q9,q10,q14,q20,q21,q22,q24,q32,q34,q35,q39,q41,continente
2264,22-24,Man,Egypt,Bachelor’s degree,1-3 years,"[Python, R, SQL, Java]",Python,"[Visual Studio , PyCharm , Spyder, Jupyter No...","[Kaggle Notebooks, Colab Notebooks, Amazon Sag...","[ Matplotlib , Seaborn ]",[Computers/Technology],0-49 employees,5-9,[Analyze and understand data to influence prod...,"Oracle Database , Google Cloud BigQuery , Goo...",[ None],NaN,"[GitHub, Kaggle, Colab]",[Basic statistical software (Microsoft Excel-G...,África
430,18-21,Man,India,Bachelor’s degree,< 1 years,"[Python, C, C++]",Python,"[Visual Studio Code (VSCode) , PyCharm , Spyd...",[Colab Notebooks],[ Matplotlib ],[Academics/Education],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Asia
1189,60-69,Man,Argentina,Master’s degree,I have never written code,NaN,NaN,NaN,NaN,NaN,[Academics/Education],0-49 employees,0,[Analyze and understand data to influence prod...,NaN,NaN,NaN,NaN,[Local development environments (RStudio-Jupyt...,América del Sur


## 4. Guardado de los datos

In [18]:
df.to_csv('../resultados/df_da_relevante_limpio.csv')
df.to_pickle('../resultados/df_da_relevante_limpio.pkl')